In [4]:
import random
import array
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing

from deap import base
from deap import creator
from deap import tools


def initIndividual(icls, size):
    """
        Initialization function for an individual
    """
    ind = icls(get_random_arrangement(size))
    return ind

N_TURB = 50
MIN_LOC = 50
MAX_LOC = 3950
CXPB = 0.5
MUTPB = 0.01
#---------
# Create
#---------
creator.create("FitnessMulti", base.Fitness, weights=(1.0,)) 
creator.create("Individual", list , fitness=creator.FitnessMulti)


toolbox = base.Toolbox()
toolbox.register("individual", initIndividual, creator.Individual,N_TURB)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# register the goal / fitness function
toolbox.register("evaluate", evaluateAEP)

# pool = multiprocessing.Pool()
toolbox.register("map", map)
# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.2, indpb=0.2)

toolbox.decorate("mate", checkBounds(MIN_LOC, MAX_LOC))
toolbox.decorate("mutate", checkBounds(MIN_LOC, MAX_LOC))
# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
# toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("select", tools.selSPEA2)


NameError: name 'evaluateAEP' is not defined

In [5]:
import numpy as np
import random
import shapely
from shapely.geometry import Point, Polygon, LineString, GeometryCollection
from shapely.ops import nearest_points 
from shapely import wkt

# from Wind_Farm_Evaluator.Vec_modified import *

def get_random_arrangement(n_turbs):

    def get_point():
        """Returns random integer from 50 to 3950 inclusive"""
        return random.uniform(50,3950)

    def is_valid(point):
        """Checks if given point is valid"""
        point = np.array(point)
        point = np.reshape(point,(1,2))
        # getting array of distances to every other point
        dist = np.linalg.norm(turbine_pos - point,axis=1)
        return min(dist) > 400   # 400 is the problem constraint

    turbine_pos = np.full((n_turbs,2),np.inf)
    turb_list = []
    count = 0
    while count < n_turbs:
        point = [get_point(),get_point()] # x,y
        if is_valid(point):
            turbine_pos[count,:] = point
            count += 1
            # turb_list.append(point)
    return turbine_pos


def parse_data(n_turbs):
    # setting turbine radius
    turb_rad = 50.0

    # Loading the power curve
    power_curve   =  loadPowerCurve('./Shell_Hackathon Dataset/power_curve.csv')

    # Loading wind data 
    years = ['07']#,'08','09','13','14','15','17']
    wind_inst_freqs = []
    for y in years:
        wind_inst_freqs.append(binWindResourceData(f'./Shell_Hackathon Dataset/Wind Data/wind_data_20{y}.csv'))
    
    # preprocessing the wind data to avoid repeated calculations
    n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing(power_curve,n_turbs)

    kwargs = {'turb_rad': turb_rad, 'power_curve': power_curve, 'wind_inst_freqs': wind_inst_freqs,
            'n_wind_instances': n_wind_instances, 'cos_dir': cos_dir, 'sin_dir': sin_dir, 'wind_sped_stacked': wind_sped_stacked,
            'C_t': C_t}

    return kwargs

def evaluateAEP(individual):#, n_turbs, turb_rad, power_curve, wind_inst_freqs, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t):
    """
        Function to return values of both objectives as a tuple
    """
    n_turbs = len(individual)
    # rearranging the terms in individual to pass to getAEP
    turb_coords = np.array([[individual[i][0],individual[i][1]] for i in range(0,n_turbs)])
    data = parse_data(n_turbs)
    # calculating meanAEP for 
    mean_AEP = 0
    for wind_inst_freq in data['wind_inst_freqs']:
        mean_AEP += getAEP(data['turb_rad'], turb_coords, data['power_curve'], wind_inst_freq, data['n_wind_instances'], data['cos_dir'], data['sin_dir'], data['wind_sped_stacked'], data['C_t'])
    mean_AEP /= len(data['wind_inst_freqs'])
    
    ideal_AEP = 11.297*n_turbs  # 11.297 is the mean score for 1 turbine
    
    return mean_AEP/ideal_AEP, # First objective should be closest to 1 and second closest to zero

def peri_constraint(turb_coords):
    peri_val = 0
    for turb in turb_coords:
        for val in turb:
            if val < 50:
                peri_val += (50 - val)
            elif val > 3950:
                peri_val += (val - 3950)
                
    return peri_val

def proxi_constraint(turb_coords):
    """
        Function to penalize if proximity contraint is violated.
        turb_coords is a 2d numpy array with N (xi,yi) elements.
    """
    proxi_val = 0
    for i in range(turb_coords.shape[0]-1):
        for j in range(i+1,turb_coords.shape[0]-1):
            norm = np.linalg.norm(turb_coords[i]-turb_coords[j])
            proxi_val += max(0,400 - norm)

    return proxi_val


def checkBounds(Min, Max):
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                field = Polygon([(Min,Min), (Min,Max), (Max,Max), (Max,Min)])
                for i,point in enumerate(child):
                    pt = Point(point)
                    if field.contains(pt):
                        pass
                    else:
                        pt_new,_ = nearest_points(field,pt)
                        pt = Point(pt_new)
                        child[i] = np.array([pt.x, pt.y])
                        # print(point)
                        # print(child[i])
                    field = field.difference(pt.buffer(400))                   
            return offspring
        return wrapper
    return decorator

In [ ]:
%%time
pop = toolbox.population(n=300)
fitnesses = list(toolbox.map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

In [ ]:
print("  Evaluated %i individuals" % len(pop))

# Extracting all the fitnesses of 
fits = [ind.fitness.values[0] for ind in pop]

# Variable keeping track of the number of generations
g = 0

# Begin the evolution
while max(fits) < 1 and g < 100:
    # A new generation
    g = g + 1
    print("-- Generation %i --" % g)
    
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(toolbox.map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):

        # cross two individuals with probability CXPB
        if random.random() < CXPB:
            toolbox.mate(child1, child2)

            # fitness values of the children
            # must be recalculated later
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:

        # mutate an individual with probability MUTPB
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    #Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(invalid_ind))
    
    # The population is entirely replaced by the offspring
    pop[:] = offspring
    
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]
    
    length = len(pop)
    mean = sum(fits) / length       
    
    print("  Min %s" % min(fits))
    print("  MaxN_TURB = 50 %s" % max(fits))
    print("  Avg %s" % mean)

print("-- End of (successful) evolution --")
best_ind = tools.selBest(pop, 1)[0]
print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))